# Ambientes interactivos
Vamos a integrar todas las herramientas que tenemos hasta ahora en un conjunto de aplicaciones utilizando las librerías de Python. La programación se pone interesante cuando el total vale más que la suma de sus partes.
___

## Canvas y gestion de eventos (bind)
Un `canvas` es un widget que permite incluír gráficos en el GUI. Se pueden incluir figuras geométricas que se definen por código e inclusive incrustar una imágen.

In [2]:
import tkinter as tk
import tkinter.ttk as ttk
from random import randrange

class App:
    def __init__(self, master):
        self.master = master
        self.master.title("Tk Animation App")
        
        self.WIDTH = 300
        self.HEIGHT = 400
        self.SIZE = 15
  
        self.x = randrange(1, self.WIDTH - self.SIZE - 1) 
        self.y = randrange(1, self.HEIGHT - self.SIZE - 1)
        self.canvas = tk.Canvas(self.master, bg='white', 
                                width=self.WIDTH, height=self.HEIGHT)
        self.canvas.pack()

        self.ball = self.canvas.create_oval(self.x, self.y, 
                                            self.x + self.SIZE, self.y + self.SIZE, 
                                            fill='red')
        self.canvas.bind("<Button-1>", self.update_graph)

        
    def update_graph(self, event):
        xmouse, ymouse = event.x, event.y
        
        if self.x < xmouse < (self.x + self.SIZE) and self.y < ymouse < (self.y + self.SIZE):
            self.canvas.delete(self.ball)
            self.x, self.y = randrange(0, self.WIDTH), randrange(0, self.HEIGHT)
            self.ball = self.canvas.create_oval(self.x, self.y, 
                                                self.x + self.SIZE, self.y + self.SIZE, 
                                                fill='red')

root = tk.Tk()
app = App(root)
root.mainloop()

## Lazo temporizado con `after`
Muchos widgets tienen un método llamado `after` que mantiene un temporizador asociado a este. Cada vez que se llama al método, se establece un cronómetro y una vez vebcido el tiempo se invoca a otro método o función.

In [1]:
import tkinter as tk
import tkinter.ttk as ttk
from random import randrange, uniform

class App:
    def __init__(self, master):
        self.master = master
        self.master.title("Tk Animation App")

        self.WIDTH = 300
        self.HEIGHT = 400
        self.SIZE = 10
        self.DELAY = 50    # 50ms
        
        self.x = randrange(1, self.WIDTH - self.SIZE - 1)
        self.y = randrange(1, self.HEIGHT - self.SIZE - 1)
        self.dx = uniform(-10, 10)
        self.dy = uniform(-10, 10)
        self.pause_animation = False
        
        self.canvas = tk.Canvas(self.master, bg='white', width=self.WIDTH, height=self.HEIGHT)
        self.canvas.pack()
        
        self.ball = self.canvas.create_oval(self.x, self.y, 
                                            self.x + self.SIZE, self.y + self.SIZE, 
                                            fill='red')

        self.canvas.bind("<Button-1>", self.control_animation)
        

    def control_animation(self, handle):
        self.pause_animation = not self.pause_animation  
        
        if self.pause_animation:
            self.text_pause = self.canvas.create_text(self.WIDTH/2, self.HEIGHT/2,
                                                      text="Animation Paused", 
                                                      font='Arial 12 bold')
        else:
            self.canvas.delete(tk.ALL)
        
        
    def animate_canvas(self):
        if not self.pause_animation:
            self.canvas.delete(self.ball)
            
            if self.x <= 0 or (self.x + self.SIZE) >= self.WIDTH:
                self.dx = -self.dx
                
            if self.y <= 0 or (self.y+ self.SIZE) >= self.HEIGHT:
                self.dy = -self.dy
                
            self.x += self.dx
            self.y += self.dy
            
            self.ball = self.canvas.create_oval(self.x, self.y, 
                                                self.x + self.SIZE, self.y + self.SIZE, 
                                                fill='red')
            
        self.canvas.after(self.DELAY, self.animate_canvas)    
          

root = tk.Tk()
app = App(root)
app.animate_canvas()
root.mainloop()

## Tiempo real en tkinter
Utilizando el método `after`, se pueden temporizar varios procesos de forma simultanea en el mismo GUI. Podemos utilizar la librería `psutil` que retorna información sobre el hardware del sistema y monitorearlo utilizando una aplicación gráfica.

In [ ]:
import psutil

print(f"Uso del CPU [%]: {psutil.cpu_percent()}")
print(f"Velocidad del CPU [Mhz]: {psutil.cpu_freq().current:,}")
print(f"Memoria instalada [Mb]: {psutil.virtual_memory().total / 1e6:,}")
print(f"Battery [%]: {psutil.sensors_battery().percent}")

In [2]:
import psutil
import tkinter as tk
import tkinter.ttk as ttk

class App:
    def __init__(self, master):
        self.master = master
        self.master.title("CPU Data")
        self.master.resizable(0, 0)

        self.var_usage_CPU = tk.DoubleVar()
        self.var_battery_percent = tk.IntVar(value=50)

        frm = tk.Frame()
        frm.pack(padx=10, pady=10)
        
        self.lblCPU_Usage = tk.Label(frm, text="", font="Arial 16 bold")
        self.lblBAT_Usage = tk.Label(frm, text="", font="Arial 16 bold")

        self.lblCPU_Usage.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.lblBAT_Usage.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)

        self.read_psutil_data()


    def read_psutil_data(self):
        self.var_usage_CPU.set(psutil.cpu_percent())
        self.var_battery_percent.set(psutil.sensors_battery().percent)
        self.lblCPU_Usage.config(text=f"CPU Usage: {self.var_usage_CPU.get()}%")
        self.lblBAT_Usage.config(text=f"Battery Available: {self.var_battery_percent.get()}%")
        
        if psutil.sensors_battery().power_plugged:
            self.lblBAT_Usage.config(fg='black');
        else:
            self.lblBAT_Usage.config(fg='red');
        
        self.master.after(1000, self.read_psutil_data)


root = tk.Tk()
app = App(root)
root.mainloop()

## matplotlib en tkinter
Se puede insertar un gráfico de matplotlib en una interface tkinter, utilizando el objeto `FigureCanvasTkAgg`. Esto permite crear una figura que recibirá el `Figure` de matplotlib y lo convertirá en un widget tipo `Canvas`.

Posteriormente, este canvas de puede actualizar con el método `draw`. Los cambios en la gráfica se hacen directamente sobre la información de los arreglos a plotear.

Podemos diseñar una aplicación que muestre la expansión de Fourier de una onda cuadrada definida como:

$$ T(t) = \begin{cases}\space \space \space 1 \space \space (t \lt t \lt T)\\ -1 \space \space (-T \lt t \lt 0)\end{cases} $$

cuya serie de Fourier esta dada por:

$$ F(t) = \frac{4}{\pi}\sum_{k=0}^\infty \frac{1}{2k+1}sin(2k+1)\pi t $$

Donde _k_ será el número de terminos de la serie, y _t_ un rango (entre -1.1 y 1.1, en pasos de 0.01)

In [ ]:
import tkinter.ttk as ttk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class App:
    def __init__(self, master):
        self.master = master
        self.master.geometry("620x320+100+100")
        self.master.resizable(0, 0)
        self.master.title("Serie de Fourier - Cuadrada")

        self.graph_colors = ['blue', 'red', 'green', 'black']
        self.var_nterm = tk.IntVar(value=1)
        self.var_color = tk.IntVar(value=0)
        self.var_grid = tk.BooleanVar(value=True)

        frmI = tk.Frame(self.master)
        frmD = tk.Frame(self.master)
        frmI.pack(side=tk.LEFT)
        frmD.pack(side=tk.LEFT)
        
        frmCol = tk.LabelFrame(frmD, text="Colores")     
        frmCol.pack(padx=10, pady=10, ipady=5, ipadx=40)

        frmTerm = tk.LabelFrame(frmD, text="Num Term")     
        frmTerm.pack(padx=10, pady=10, ipadx=5)
        
        self.fig, self.ax = plt.subplots(figsize=(6, 4), facecolor="#F0F0ED")
        t, y = self.fourier_cuad(self.var_nterm.get())
        self.line, = self.ax.plot(t, y)
        self.ax.grid(self.var_grid.get())
        self.ax.set_ylim(-1.5, 1.5)

        # -------------------- frmI ------------------------------
        self.graph = FigureCanvasTkAgg(self.fig, master=frmI)
        self.graph.get_tk_widget().pack(expand=True, fill=tk.X)

        # -------------------- frmD ------------------------------
        self.rdoAzul = tk.Radiobutton(frmCol, text="Azul", variable=self.var_color, value=0, command=self.set_line_color)
        self.rdoRojo = tk.Radiobutton(frmCol, text="Rojo", variable=self.var_color, value=1, command=self.set_line_color)
        self.rdoVerde = tk.Radiobutton(frmCol, text="Verde", variable=self.var_color, value=2, command=self.set_line_color)
        self.rdoNegro = tk.Radiobutton(frmCol, text="Negro", variable=self.var_color, value=3, command=self.set_line_color)
        
        self.rdoAzul.grid(row=0, column=0, sticky=tk.W)
        self.rdoRojo.grid(row=1, column=0, sticky=tk.W)
        self.rdoVerde.grid(row=2, column=0, sticky=tk.W)
        self.rdoNegro.grid(row=3, column=0, sticky=tk.W)
        
        self.sclTerminos = ttk.Scale(frmTerm, from_=1, to=25, variable=self.var_nterm, command=self.printScale)
        self.lblNumTerminos = tk.Label(frmTerm, text="{:2}".format(self.var_nterm.get()))
        self.sclTerminos.grid(row=0, column=0, padx=5, pady=5)
        self.lblNumTerminos.grid(row=0, column=1)

        self.chkGrid = tk.Checkbutton(frmD, text="Mostrar grilla?", variable=self.var_grid, command=self.show_grid)
        self.chkGrid.pack()
        
        
    def fourier_cuad(self, n):
        t = np.arange(-1.1, 1.1, 0.01)
        F = np.zeros(t.size)    

        for k in range(n):
            # Se va acumulando la sumatoria a cada uno de los terminos de F
            F += (1 / (2 * k + 1)) * np.sin((2 * k + 1) * np.pi * t)

        return t, (4 / np.pi) * F

    
    def printScale(self, n_term):
        self.lblNumTerminos.config(text="{:2}".format(self.var_nterm.get()))
        self.line.set_ydata(self.fourier_cuad(self.var_nterm.get())[1])  # [1]: Solo componente y
        self.graph.draw()

        
    def set_line_color(self):
        self.line.set_color(self.graph_colors[self.var_color.get()])
        self.graph.draw()
        
        
    def show_grid(self):
        self.ax.grid(self.var_grid.get())
        self.graph.draw()
        
        
root = tk.Tk()
app = App(root)
root.mainloop()

## ipywidgets: interactividad en un Jupyter Notebook
Los Jupyter Notebooks también soportan widgets gráficos con la combinación de las librerías `ipywidgets` y `IPython.display`. `ipywidgets` contiene una colección de objetos gráficos contstruidos con JavaScript que se pueden ejecutar en el documento. `IPython.display` es una colección de *renders* que permiten mostrar información gráfica.

In [ ]:
from IPython.display import display
from ipywidgets import IntSlider

slider = IntSlider(min=10, max=100, step=5)
display(slider)

Los valores gestionados por estos widgets (`widget.value`) se pueden asociar como parametros de una función con `interact`:

In [ ]:
from ipywidgets import interact

def suma(a, b):
    print("Suma:", a + b)
    
gui = interact(suma, a=(1, 10), b=(1, 10))
display(gui)

Otra forma de hacer esto es utilizando `interact` como un decorador de la función:

In [ ]:
@interact(a=(1, 10), b=(1, 10))
def suma(a, b):
    print("Suma:", a + b)

### Algunos widgets disponibles
Lista completa: (https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)

In [ ]:
from ipywidgets import *

w1 = IntSlider(description="IntSlider")
w2 = FloatSlider(description="FloatSlider")
w3 = BoundedIntText(description="BoundedIntText")
w4 = BoundedFloatText(description="BoundedFloatText", step=0.5)
w5 = Combobox(description="Combobox", options=['1', '2', '3'])
w6 = Dropdown(description="Dropdown", options=[1, 2, 3])
w7 = RadioButtons(description="RadioButtons", options=[1, 2, 3])
w8 = Checkbox(description="Checkbox")
w9 = Text(description="Text")
w10 = Button(description="Button")

gui = VBox([w1, w2, w3, w4, w5, w6, w7, w8, w9, w10])

display(gui)

## Eventos
Los widgets manejan eventos definidos como métodos del objeto widget. Por ejemplo, en el caso del `Button`, el evento a consutar en `on_click`:

In [ ]:
txt = Text(description="Nombre")
but = Button(description="Imprime")

gui = HBox([txt, but])
display(gui)

def print_name(event):
    print(txt.value)

but.on_click(print_name)

Otros generan eventos al realizar cambios. La mejora manera de tomar control sobre estas acciones es a través de `interactive`, que retorna un VBox que se puede instanciar como un objeto para luego mostrarlo con ``display`:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_circle(r, color, grid, line_style):
    x = r * np.cos(np.linspace(0, 2*np.pi, 100))
    y = r * np.sin(np.linspace(0, 2*np.pi, 100))
    plt.plot(x, y, color=color, linestyle=line_style)
    plt.axis('square')
    plt.grid(grid)
    plt.xlim(-12, 12)
    plt.ylim(-12, 12)
    plt.show()
    
w1 = FloatSlider(description="Radio:", min=1, max=12)  
w2 = Dropdown(description="Color", options={'Azul': 'blue', 'Rojo': 'red', 'Verde': 'green'})
w3 = Checkbox(description="Grilla?")
w4 = RadioButtons(description="Linea", options=["-", "--", "-.", ":"])

gui = interactive(plot_circle, r=w1, color=w2, grid=w3, line_style=w4, continuous_update=False)
display(gui)

Si se quiere organizar los diferentes widgets en el documento, hay que agruparlos en `Hbox` y `VBox`. Sin embargo, es necesario asociar los widgets a la función con `interactive`, que retorna un `VBox`, por lo que si se muestran ambos con `display` se verán ambos diseños. Existe el objeto `Layout` y algunos métodos de diseño (como `Grid` y `FlexBox`), pero podemos salir del paso utilizando algunos tips:

### Tip: Como crear un documento Jupyter interactivo con diseño

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display

# Funcion que sera llamada de forma interactiva
def plot_circle(r, color, grid, line_style):
    x = r * np.cos(np.linspace(0, 2*np.pi, 100))
    y = r * np.sin(np.linspace(0, 2*np.pi, 100))
    plt.plot(x, y, color=color, linestyle=line_style)
    plt.axis('square')
    plt.grid(grid)
    plt.xlim(-12, 12)
    plt.ylim(-12, 12)
    plt.show()
    
# Definicion de widgets
w1 = FloatSlider(description="Radio:", min=1, max=12)  
w2 = Dropdown(description="Color", options={'Azul': 'blue', 'Rojo': 'red', 'Verde': 'green'})
w3 = Checkbox(description="Grilla?")
w4 = RadioButtons(description="Linea", options=["-", "--", "-.", ":"])

# Organizacion de widgets en HBOx y VBox
box1 = HBox([w1, w2])
box2 = HBox([w3, w4]) 
layout = VBox([box1, box2])

# Definicion de los controles (interactive)
controls = interactive(plot_circle, r=w1, color=w2, grid=w3, line_style=w4, continuous_update=False)

# El ultimo elemento de interactive tiene el resultado de los widgets (el llamado a la función)
output = controls.children[-1]

# Asignamos un valor inicial a un widget para que llame a la función
w1.value=6

# Se muestra el ambiente interactivo
display(HBox([layout, output]))

## Y Voila!
En el 2019 [se publico en GitHub el código del proyecto Voila](https://github.com/voila-dashboards/voila), que promete convertír un documento Jupyter en una aplicación web interactiva. Esto permite tener un Jupyter Notebook como un dashboard web con los resultados de las celdas, pero lo interesante sucede cuando el documento contiene widgets interactivos, pues estos son generados como parte del código.

Primero, instale Voila:

    conda install -c conda-forge voila
    
Luego, reinicie el ambiente Jupyter y podrá ver un nuevo icono en la bara de herramientas, en la sección central. Confeccione un documento Jupyter que contenga solamente una celda (por ejemplo, copie toda la celda anterior en un solo documento) y luego haga click en el nuevo boton.

Y Voila... ¡un dashboard web sin haber escrito una sola letra de JavaScript! ¿Se imagina lo que puede construír?